Import Library

In [1]:
import pandas as pd
import multiprocessing
import time

Open Data

In [4]:
df = pd.read_csv('../../train.csv')
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


Take 25% of the Data

In [5]:
df_sample = df.sample(frac=0.25, random_state=42)
sampled = df_sample["trip_duration"]

Sorting Using Multiprocessing

In [6]:
# Split the DataFrame into chunks
def split_dataframe(df, n_chunks):
    chunk_size = len(df) // n_chunks
    return [df[i*chunk_size:(i+1)*chunk_size] for i in range(n_chunks - 1)] + [df[(n_chunks - 1)*chunk_size:]]

# Each process sorts one chunk
def sort_chunk(chunk):
    return chunk.sort_values(by='trip_duration')

# Final merge (simple version using re-sorting)
def merge_sorted_chunks(sorted_chunks):
    return pd.concat(sorted_chunks).sort_values(by='trip_duration').reset_index(drop=True)


Split the Data for Multiprocecssing

In [9]:
num_processes = 4
chunks = split_dataframe(sampled, num_processes)
chunks

[67250      1040
 1397036     827
 1021087     614
 951424      867
 707882     4967
            ... 
 9060        901
 488454      437
 286427     1189
 697976     1207
 967251     1004
 Name: trip_duration, Length: 91165, dtype: int64,
 122024     1880
 536602     1009
 16993       226
 374350      308
 1136622     589
            ... 
 1071110     691
 1202750    1773
 404713      503
 778402      243
 335208      218
 Name: trip_duration, Length: 91165, dtype: int64,
 1224558     614
 579001      611
 86830       738
 413078      252
 909557     1170
            ... 
 101145      905
 195601      414
 626060      527
 406364      145
 37713       326
 Name: trip_duration, Length: 91165, dtype: int64,
 68621       444
 1094878     495
 198855     1179
 73691       581
 1110940    1035
            ... 
 170051      114
 500460      788
 338334      288
 388537      418
 501361     1414
 Name: trip_duration, Length: 91166, dtype: int64]

Run MultiProcessing

In [ ]:
with multiprocessing.Pool(processes=num_processes) as pool:
    sorted_chunks = pool.map(sort_chunk, chunks)

sorted_df = merge_sorted_chunks(sorted_chunks)